In [144]:
import sqlite3 as sql
import pandas  as pd
import numpy as np
#table_name = "dataset"
#read_data() for getting data from dataset
#get() for getting sequence,mutation,ddG for which we should make the prediction
#output() for sending result to server
#file_name = name_of_file.sqlite
def inverse_mutation(line):
    sequence,current,position,mutation,ddG,PH,Temp = line
    current,mutation = mutation,current
    sequence = sequence[:int(position)]+current+sequence[int(position)+1:]
    ddG = str(-float(ddG))
    return [sequence,current,position,mutation,ddG,PH,Temp]
def read_data(file_name):
    conn = sql.connect(file_name)
    cursor = conn.cursor()
    types = [str,str,int,str,float,float,float]
    names = ["sequence","current","position","mutation","ddG","PH","Temp"]
    results = {names[i]:[] for i in range(len(names))}
    for row in cursor.execute("SELECT * FROM dataset"):
        row = list(row)
        row[2] = str(int(row[2])-1)
        inversed = inverse_mutation(row)
        for i in range(len(names)):
            results[names[i]].append(types[i](row[i]))
            results[names[i]].append(types[i](inversed[i]))
    result = pd.DataFrame(results)
    conn.close()
    return result
def get():
    text = raw_input()
    types = [str,str,int,str,float]
    split_text = text.split(" ")
    return [types[i](split_text[i]) for i in range(len(split_text))]
#values = [sequence,current,position,mutation,ddG]
def output(values):
    result = values[0]
    for i in range(1,len(values)):
        result += " "+ str(values[i])
    print (result)

In [83]:
from sklearn.preprocessing import MinMaxScaler

In [204]:
raw_data = read_data("iwdb.sqlite")
raw_data.head()

,PH,Temp,current,ddG,mutation,position,sequence
0,7.4,25.0,L,-1.18,K,13,AAGHVNIAEAVQQLNHTIVNAAHELHETLGLPTPDEALNLLTEQAN...
1,7.4,25.0,K,1.18,L,13,AAGHVNIAEAVQQKNHTIVNAAHELHETLGLPTPDEALNLLTEQAN...
2,8.5,20.0,L,0.81,S,36,MEPAAGSSMEPSADWLATAAARGRVEEVRALLEAGALPNAPNSYGR...
3,8.5,20.0,S,-0.81,L,36,MEPAAGSSMEPSADWLATAAARGRVEEVRALLEAGASPNAPNSYGR...
4,7.4,25.0,A,-1.40,K,136,AAGHVNIAEAVQQLNHTIVNAAHELHETLGLPTPDEALNLLTEQAN...


In [205]:
raw_data[['sequence','position']].head()

,sequence,position
0,AAGHVNIAEAVQQLNHTIVNAAHELHETLGLPTPDEALNLLTEQAN...,13
1,AAGHVNIAEAVQQKNHTIVNAAHELHETLGLPTPDEALNLLTEQAN...,13
2,MEPAAGSSMEPSADWLATAAARGRVEEVRALLEAGALPNAPNSYGR...,36
3,MEPAAGSSMEPSADWLATAAARGRVEEVRALLEAGASPNAPNSYGR...,36
4,AAGHVNIAEAVQQLNHTIVNAAHELHETLGLPTPDEALNLLTEQAN...,136


In [206]:
def neighbour(sequence,position,shift):
    if (position + shift > len(sequence)-1 or position + shift < 0):
        return "0"
    else:
        return sequence[position+shift]

In [207]:
raw_data['Left'] = raw_data.apply(lambda row: neighbour(row['sequence'],row['position'],-1),axis=1)
raw_data['Right'] = raw_data.apply(lambda row: neighbour(row['sequence'],row['position'],1),axis=1)
raw_data['LeftLeft'] = raw_data.apply(lambda row: neighbour(row['sequence'],row['position'],-2),axis=1)
raw_data['RightRight'] = raw_data.apply(lambda row: neighbour(row['sequence'],row['position'],2) ,axis=1)
raw_data.head()

,PH,Temp,current,ddG,mutation,position,sequence,Left,Right,LeftLeft,RightRight
0,7.4,25.0,L,-1.18,K,13,AAGHVNIAEAVQQLNHTIVNAAHELHETLGLPTPDEALNLLTEQAN...,Q,N,Q,H
1,7.4,25.0,K,1.18,L,13,AAGHVNIAEAVQQKNHTIVNAAHELHETLGLPTPDEALNLLTEQAN...,Q,N,Q,H
2,8.5,20.0,L,0.81,S,36,MEPAAGSSMEPSADWLATAAARGRVEEVRALLEAGALPNAPNSYGR...,A,P,G,N
3,8.5,20.0,S,-0.81,L,36,MEPAAGSSMEPSADWLATAAARGRVEEVRALLEAGASPNAPNSYGR...,A,P,G,N
4,7.4,25.0,A,-1.40,K,136,AAGHVNIAEAVQQLNHTIVNAAHELHETLGLPTPDEALNLLTEQAN...,E,A,Q,E


In [208]:
raw_data = raw_data.sample(frac=1).reset_index(drop=True)
raw_data.head()

,PH,Temp,current,ddG,mutation,position,sequence,Left,Right,LeftLeft,RightRight
0,7.0,20.0,A,1.69,T,21,ATSTKKLHKEPATLIKAIDGDAVKLMYKGQPMTFRLLLVDTPETKH...,D,V,G,K
1,7.0,20.0,R,-1.64,A,125,ATSTKKLHKEPATLIKAIDGDTVKLMYKGQPMTFRLLLVDTPETKH...,L,K,H,S
2,7.0,25.0,G,0.50,H,9,AKLNESHRLGQFFRDMDDEESWIKEKKLLVSSEDYGRDLTGVQNLR...,L,Q,R,F
3,2.7,65.6,G,-1.46,A,116,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...,K,T,C,D
4,6.0,10.0,L,1.82,V,18,MELKHSISDYTEAEFLQLLTTICNADTSSEEELVKLVTHFEEMTEH...,L,T,Q,T


In [200]:
inputting = raw_input()

A B C D E F G H I J K L M N O P Q R S T U V W X Y Z 0


In [209]:
symbols = inputting.split(' ')
numerics = [i for i in range(len(symbols))]

In [210]:
categorical = ['current','mutation','Left','Right','LeftLeft','RightRight']
for item in categorical:
    raw_data[item] = raw_data[item].apply(lambda x: numerics[symbols.index(x)])

In [211]:
raw_data.head()

,PH,Temp,current,ddG,mutation,position,sequence,Left,Right,LeftLeft,RightRight
0,7.0,20.0,0,1.69,19,21,ATSTKKLHKEPATLIKAIDGDAVKLMYKGQPMTFRLLLVDTPETKH...,3,21,6,10
1,7.0,20.0,17,-1.64,0,125,ATSTKKLHKEPATLIKAIDGDTVKLMYKGQPMTFRLLLVDTPETKH...,11,10,7,18
2,7.0,25.0,6,0.50,7,9,AKLNESHRLGQFFRDMDDEESWIKEKKLLVSSEDYGRDLTGVQNLR...,11,16,17,5
3,2.7,65.6,6,-1.46,0,116,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...,10,19,2,3
4,6.0,10.0,11,1.82,21,18,MELKHSISDYTEAEFLQLLTTICNADTSSEEELVKLVTHFEEMTEH...,11,19,16,19


In [212]:
train_data = raw_data.iloc[:3680]
test_data = raw_data.iloc[3680:]
raw_data.shape,train_data.shape,test_data.shape

((4422, 11), (3680, 11), (742, 11))

In [213]:
train_labels = train_data['ddG'].values

In [214]:
train_data = train_data.drop(['ddG','sequence'],axis=1)

In [215]:
train_data.head()

,PH,Temp,current,mutation,position,Left,Right,LeftLeft,RightRight
0,7.0,20.0,0,19,21,3,21,6,10
1,7.0,20.0,17,0,125,11,10,7,18
2,7.0,25.0,6,7,9,11,16,17,5
3,2.7,65.6,6,0,116,10,19,2,3
4,6.0,10.0,11,21,18,11,19,16,19


In [216]:
train_labels

array([ 1.69, -1.64,  0.5 , ..., -0.  ,  1.22, -1.81])

In [217]:
test_labels = test_data['ddG'].values

In [218]:
test_data = test_data.drop(['ddG','sequence'],axis=1)

In [219]:
test_data.head()

,PH,Temp,current,mutation,position,Left,Right,LeftLeft,RightRight
3680,7.4,25.0,18,0,121,12,0,18,3
3681,7.8,23.0,5,22,257,21,21,10,16
3682,6.3,25.0,7,16,17,24,10,19,11
3683,6.5,25.0,19,11,8,2,11,13,17
3684,5.5,20.0,5,24,35,19,8,13,24


In [223]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn import pipeline
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics

In [220]:
binary_data_columns = ['holiday', 'workingday']
binary_data_indices = np.array([(column in binary_data_columns) for column in train_data.columns], dtype = bool)
categorical_data_columns = ['current', 'mutation', 'Left','Right','LeftLeft','RightRight'] 
categorical_data_indices = np.array([(column in categorical_data_columns) for column in train_data.columns], dtype = bool)
numeric_data_columns = ['Temp', 'PH', 'position']
numeric_data_indices = np.array([(column in numeric_data_columns) for column in train_data.columns], dtype = bool)
categorical_data_indices

array([False, False,  True,  True, False,  True,  True,  True,  True])

In [225]:
regr = AdaBoostRegressor(random_state=0, n_estimators=100)
estimator = pipeline.Pipeline(steps = [       
    ('feature_processing', pipeline.FeatureUnion(transformer_list = [        
            #binary
            ('binary_variables_processing', preprocessing.FunctionTransformer(lambda data: data[:, binary_data_indices])), 
                    
            #numeric
            ('numeric_variables_processing', pipeline.Pipeline(steps = [
                ('selecting', preprocessing.FunctionTransformer(lambda data: data[:, numeric_data_indices]))
                        ])),
        
            #categorical
            ('categorical_variables_processing', pipeline.Pipeline(steps = [
                ('selecting', preprocessing.FunctionTransformer(lambda data: data[:, categorical_data_indices])),
                ('hot_encoding', preprocessing.OneHotEncoder(handle_unknown = 'ignore'))            
                        ])),
        ])),
    ('model_fitting', regr)
    ]
)

In [226]:
estimator.fit(train_data,train_labels)

Pipeline(memory=None,
     steps=[('feature_processing', FeatureUnion(n_jobs=None,
       transformer_list=[('binary_variables_processing', FunctionTransformer(accept_sparse=False, check_inverse=True,
          func=<function <lambda> at 0x000000000EB33908>, inv_kw_args=None,
          inverse_func=None, kw_args=None, pass_y=...(base_estimator=None, learning_rate=1.0, loss='linear',
         n_estimators=100, random_state=0))])

In [227]:
metrics.mean_absolute_error(test_labels, estimator.predict(test_data))

1.27790245135116